In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Data Loading

Мы имеем 4 практически идентичных формата датасетов на портале sfdata.org за исключением наименования колонок и уникального идентификатора. Порядковый номер увеличивается по мере старения формата. 1 - самый новый, 4 самый старый.

Последний отличается от старых форматом идентификаторов объектов, но по факту в новом версии они просто имеют префикс "APN" записанное в колонку "APN", а в старой это число, которое иногда имеет префикс "0" и записано в колонках "BLKLOT" или "Block Lot". В данных до 3 квартала 2014 отсутствует информация об affordable housing.

Для анализа использовались данные только со статусом CONSTRUCTION в колонках "BESTSTAT" (или аналогичном), которые имеют указанное количество NET_UNITS и UNITS большее или равное нулю.

Далее некоторые пермиты не указаны и дальнейшее исследование показало что это большие сложные проекты (вроде создания новых улиц) и мы добавим к статистике позже. На данном этапе просто выведем все проекты без пермитов и удалим их из датасета.

In [2]:
def loadData(fileName, label, fmt = 1):
    columns = ['UNITS','NET_UNITS', 'AFF_UNITS', 'NET_AFF_UNITS', 'NAMEADDR', 'APN']
    
    converters = { 'APN': lambda x: x[4:], 'BP_APPLNO': lambda x: 'N'+x.replace(',','') }
    if fmt == 2 or fmt == 3:
        converters = { 'BLKLOT': lambda x: x.lstrip('0'), 'BP_APPLNO': lambda x: 'N'+x.replace(',','') }
    if fmt == 4:
        converters = { 'Block Lot': lambda x: x.lstrip('0'), 'DBI Permit': lambda x: 'N'+x.replace(',',''),
                     'Location 1': lambda x: x.split('\n')[0]}
        
    X = pd.read_csv(fileName, sep=',', parse_dates=[], infer_datetime_format=True, quotechar='"', converters=converters)
    
    if fmt == 2 or fmt == 3:
        X = X.rename(columns={"AFF_UNITS_NET": "NET_AFF_UNITS", "BLKLOT": "APN"})
    if fmt == 3:
        X = X.rename(columns={"UNITSNET": "NET_UNITS"})
    if fmt == 4:
        X = X.rename(columns={"Units": "UNITS", "Net Added Units": "NET_UNITS", "Best Stat": "BESTSTAT", 'Location 1': 'NAMEADDR', 'Block Lot': 'APN',
                             'DBI Permit': 'BP_APPLNO'})
        X['NET_AFF_UNITS'] = 0
        X['AFF_UNITS'] = 0
    if fmt == 5:
        X = X.rename(columns={"PROPUSE": "PROJECT_TYPE"})
    X = X[X['BESTSTAT'] == 'CONSTRUCTION']
    X = X[X['UNITS'] > 0]
    X = X[X['NET_UNITS'] >= 0]
    
    # Filtering records without permits
    M = X[(X['BP_APPLNO'] == 'N') | (X['BP_APPLNO'] == 'NMULTIPLE')][columns+['BP_APPLNO']]
    X = X[(X['BP_APPLNO'] != 'N') & (X['BP_APPLNO'] != 'NMULTIPLE')]
    
    # Filtering duplicated records
    D = X[X.duplicated('BP_APPLNO', keep=False)][columns+['BP_APPLNO']].sort_values('BP_APPLNO')
    X=X[~X.duplicated('BP_APPLNO')]
    
    # Set unique index by Permit ID
    X = X.set_index('BP_APPLNO')
    X.index.names = ['PERMIT_ID']
    
    X = X[columns]
    
    return X,M,D

# Helper Indexes

y13q4 = 0
y14q1 = 1
y14q2 = 2
y14q3 = 3
y14q4 = 4
y15q1 = 5
y15q2 = 6
y15q3 = 7
y15q4 = 8
y16q1 = 9
y16q2 = 10
y16q3 = 11
y16q4 = 12
y17q1 = 13

# Files and versions

files = [
    {'label': '2013\'Q4', 'file': 'data/San_Francisco_Development_Pipeline_2013_Quarter_4.csv', 'format': 4},
    {'label': '2014\'Q1', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_1.csv', 'format': 4},
    {'label': '2014\'Q2', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_2.csv', 'format': 4},
    {'label': '2014\'Q3', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_3.csv', 'format': 4},
    {'label': '2014\'Q4', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_4.csv', 'format': 3},
    {'label': '2015\'Q1', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_1.csv', 'format': 2},
    {'label': '2015\'Q2', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_2.csv', 'format': 1},
    {'label': '2015\'Q3', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_3.csv', 'format': 1},
    {'label': '2015\'Q4', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_4.csv', 'format': 1},
    {'label': '2016\'Q1', 'file': 'data/SF_Development_Pipeline_2016_Q1.csv', 'format': 1},
    {'label': '2016\'Q2', 'file': 'data/SF_Development_Pipeline_2016_Q2.csv', 'format': 5},
    {'label': '2016\'Q3', 'file': 'data/SF_Development_Pipeline_2016_Q3.csv', 'format': 1},
    {'label': '2016\'Q4', 'file': 'data/SF_Development_Pipeline_2016_Q4.csv', 'format': 1},
    {'label': '2017\'Q1', 'file': 'data/SF_Development_Pipeline_2017_Q1.csv', 'format': 1}
]

# Loading Data
count = len(files)
labels = []
data = []
missing = []
duplicates = []

for f in files:
    t,m,d = loadData(f['file'], f['label'], f['format'])
    data.append(t)
    missing.append(m)
    duplicates.append(d)
    labels.append(f['label'])

# Displaying duplicated records
Здесь мы выведем все дубликаты для дальнейшей ручной обработки

In [3]:
for i in range(count):
    if (len(duplicates[i]) > 0):
        print()
        print("Duplicated records at {}".format(labels[i]))
        display(duplicates[i])


Duplicated records at 2013'Q4


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
188,2.0,1.0,0,0,1076 Hampshire St,4152016,N200709193092
515,2.0,2.0,0,0,1078 Hampshire St,4152046,N200709193092
299,2.0,1.0,0,0,268 Madison St,5943008,N200711077587
520,1.0,1.0,0,0,268 Madison St,5943051,N200711077587



Duplicated records at 2014'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
124,7.0,7.0,0,0,4132 Third Street,5260002,N200707055953
657,7.0,7.0,0,0,4132 03rd St,5260003,N200707055953
60,2.0,1.0,0,0,1076 Hampshire St,4152016,N200709193092
627,2.0,2.0,0,0,1078 Hampshire St,4152046,N200709193092
151,2.0,1.0,0,0,268 Madison St,5943008,N200711077587
748,1.0,1.0,0,0,268 Madison St,5943051,N200711077587



Duplicated records at 2014'Q2


...

,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
236,7.0,7.0,0,0,4132 Third Street,5260002,N200707055953
514,7.0,7.0,0,0,4132 03rd St,5260003,N200707055953
171,2.0,1.0,0,0,1076 Hampshire St,4152016,N200709193092
654,2.0,2.0,0,0,1078 Hampshire St,4152046,N200709193092
112,1.0,1.0,0,0,83 Panorama Dr,2821010,N200711077576
668,1.0,1.0,0,0,83 Panorama Dr,2821023,N200711077576
269,2.0,1.0,0,0,268 Madison St,5943008,N200711077587
659,1.0,1.0,0,0,268 Madison St,5943051,N200711077587
326,2.0,2.0,0,0,447 Linden St,818048,N200912304034
651,2.0,2.0,0,0,443 Linden St,818049,N200912304034



Duplicated records at 2014'Q4


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
411,2,1,0,0,1076 HAMPSHIRE ST,4152016,N200709193092
448,2,2,0,0,1078 HAMPSHIRE ST,4152046,N200709193092
400,2,2,0,0,447 LINDEN ST,818048,N200912304034
409,2,2,0,0,443 LINDEN ST,818049,N200912304034
69,98,98,0,0,1239 TURK ST,757027,N201207104447
70,98,98,98,98,1100 GOLDEN GATE AV,757025,N201207104447
38,191,191,160,160,218 BUCHANAN ST,857001A,N201209059006
55,133,133,160,160,55 LAGUNA STREET,857001,N201209059006
53,139,139,0,0,555 FULTON ST,794028,N201305036062
54,136,136,0,0,746 LAGUNA ST,794015,N201305036062



Duplicated records at 2015'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
794,2,2,0,0,1078 HAMPSHIRE ST,4152046,N200709193092
863,2,1,0,0,1076 HAMPSHIRE ST,4152016,N200709193092
523,2,2,0,0,443 LINDEN ST,818049,N200912304034
782,2,2,0,0,447 LINDEN ST,818048,N200912304034


# Fixing Duplicates
На этом шаге мы добавляем отфильтрованные дубликаты обратно в датасет с правильными значениями.

In [4]:
def fix_duplicate(permitId, units, netUnits, affUnits, netAffUnits, addr, apn):
    for i in range(count):
        if len(duplicates[i][duplicates[i]['BP_APPLNO'] == permitId]) > 0:
            data[i].loc[permitId] = [units, netUnits, affUnits, netAffUnits, addr, apn]
        duplicates[i] = duplicates[i][duplicates[i]['BP_APPLNO'] != permitId]

fix_duplicate('N200709193092', 2, 1, 0, 0, '1076-1078 Hampshire St', '4152016')
fix_duplicate('N200711077587', 2, 1, 0, 0, '268 Madison St', '5943008')
fix_duplicate('N200707055953', 7, 7, 0, 0, '4132 03rd St', '5260003')
fix_duplicate('N200912304034', 2, 2, 0, 0, '447 Linden St', '818048')
fix_duplicate('N201209059006', 191, 191, 0, 0, '218 Buchanan St', '857001')
fix_duplicate('N200711077576', 1, 1, 0, 0, '83 Panorama Dr', '2821010')
fix_duplicate('N201207104447', 98, 98, 0, 0, '1100 GOLDEN GATE AV', '757025')
fix_duplicate('N201305036062', 139, 139, 0, 0, '555 FULTON ST', '794028')

hasDuplicates = False
for i in range(count):
    if (len(duplicates[i]) > 0):
        print()
        print("Duplicated records at {}".format(labels[i]))
        display(duplicates[i])
        hasDuplicates = True
if not hasDuplicates:
    print("No duplicates present!")

No duplicates present!


# Displaying records without Permit ID
Здесь мы выведем все записи без Permit ID

In [5]:
for i in range(count):
    if (len(missing[i]) > 0):
        print()
        print("Records without Permit ID at {}".format(labels[i]))
        display(missing[i])


Records without Permit ID at 2015'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
648,132,14,0,0,833-881 Jamestown,4991277,N



Records without Permit ID at 2015'Q3


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
48,196,196,0,0,800 BROTHERHOOD WAY,7331003,N



Records without Permit ID at 2015'Q4


...

,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
1138,54,54,0,0,800 BROTHERHOOD WAY (UNDER CONSTRUCTION),7331005,N
1261,167,167,0,0,HP SHIPYARD PHASE 1 (UNDER CONSTRUCTION),4591C001,N



Records without Permit ID at 2016'Q4


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
1179,49,49,0,0,800 BROTHERHOOD WAY (UNDER CONSTRUCTION),7331003,N
1221,110,110,0,0,HUNTERS POINT SHIPYARD (UNDER CONSTRUCTION),4591C001,NMULTIPLE



Records without Permit ID at 2017'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BP_APPLNO
1203,36,36,0,0,800 BROTHERHOOD WAY (UNDER CONSTRUCTION),7331003,NMULTIPLE
1337,86,86,0,0,HUNTERS POINT SHIPYARD (UNDER CONSTRUCTION),4591C001,NMULTIPLE
1338,107,107,106,106,HUNTERS VIEW,4624031,NMULTIPLE


# Fixing records without Permit Id
Исправляем все записи без указанного Permit Id

In [6]:
def fix_missing(permitId, units, netUnits, affUnits, netAffUnits, addr, apn):
    for i in range(count):
        if len(missing[i][missing[i]['APN'].isin(apn)]) > 0:
            data[i].loc[permitId] = [units, netUnits, affUnits, netAffUnits, addr, apn[0]]
        missing[i] = missing[i][~missing[i]['APN'].isin(apn)]
        
fix_missing("N_CUSTOM_JAMESTOWN", 132, 14, 0, 0, '833-881 Jamestown', ['4991277'])
fix_missing("N_CUSTOM_BROTHERHOOD", 196, 196, 0, 0, '800 BROTHERHOOD WAY', ['7331003', '7331005'])
fix_missing("N_CUSTOM_SHIPYARD", 229, 229, 0, 0, 'HUNTERS POINT SHIPYARD, PHASE I', ['4591C001', '4624031'])

hasMissing = False
for i in range(count):
    if (len(missing[i]) > 0):
        print()
        print("Records without Permit ID at {}".format(labels[i]))
        display(missing[i])
        hasMissing = True
if not hasMissing:
    print("No missing present")

No missing present


# Searching for invalid Unit values
Поиск неверных значений количеств юнитов в пайплайне - непостоянные значения для каждого пермита, units > net units, etc.

In [7]:
def find_incorrect():
    incorrect = {}
    for i in range(count-1):
        X = data[i]
        for j in range(i+1, count):
            X2 = data[j]
        
            X = X[X.index.isin(X2.index)].sort_index()
            X2 = X2[X2.index.isin(X.index)].sort_index()
            Y = X[X['UNITS'] != X2['UNITS']]
            Y2 = X2[X['UNITS'] != X2['UNITS']]
            if len(Y)>0:
                for index, row in Y.iterrows():
                    a = int(Y.loc[index]['UNITS'])
                    b = int(Y2.loc[index]['UNITS'])
                    if index not in incorrect:
                        incorrect[index] = { 'min': min(a, b), 'max': max(a,b)}
                    else:
                        incorrect[index] = { 'min': min(incorrect[index]['min'], a, b), 'max': max(incorrect[index]['max'], a, b)}
    return incorrect


incorrect = find_incorrect()
print("Unit number fluctuation")
display(incorrect)

Unit number fluctuation


{'N200507208144': {'max': 25, 'min': 12},
 'N200605161774': {'max': 452, 'min': 447},
 'N200711077587': {'max': 80, 'min': 1},
 'N200807176988': {'max': 2, 'min': 1},
 'N200810315586': {'max': 156, 'min': 121},
 'N200810315636': {'max': 15, 'min': 13},
 'N201009140800': {'max': 320, 'min': 305},
 'N201012237367': {'max': 39, 'min': 23},
 'N201112070227': {'max': 470, 'min': 65},
 'N201204168406': {'max': 83, 'min': 50},
 'N201207124717': {'max': 806, 'min': 285},
 'N201209059006': {'max': 450, 'min': 191},
 'N201209069080': {'max': 550, 'min': 540},
 'N201209119428': {'max': 210, 'min': 4},
 'N201211073775': {'max': 271, 'min': 121},
 'N201211093966': {'max': 84, 'min': 81},
 'N201211284953': {'max': 28, 'min': 11},
 'N201212216752': {'max': 190, 'min': 167},
 'N201303273113': {'max': 75, 'min': 69},
 'N201305015894': {'max': 140, 'min': 74},
 'N201306250394': {'max': 560, 'min': 239},
 'N201306250465': {'max': 84, 'min': 77},
 'N201306280783': {'max': 132, 'min': 4},
 'N201307303137':

In [8]:
def fix_units(permitId, units):
    for i in range(count):
        if permitId in data[i].index:
            d = data[i].loc[permitId]
            d['UNITS'] = units
            data[i].loc[permitId] = d

for k in incorrect:
    fix_units(k, incorrect[k]['max'])
    
incorrect = find_incorrect()
print("Unit number fluctuation")
display(incorrect)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Unit number fluctuation


{}

# Detection of completed buildings, p.1
Для поиска завершенных строений мы искали в следующем квартале отсутствующие записи о строительстве сравнивая их по идентификатору стройки.

In [9]:
def buildStats(data):
    buildings = list(map(lambda x: len(x), completed))
    units = list(map(lambda x: x['UNITS'].values.sum(), completed))
    netUnits = list(map(lambda x: x['NET_UNITS'].values.sum(), completed))
    aff = list(map(lambda x: x['AFF_UNITS'].values.sum(), completed[4:]))
    netAff = list(map(lambda x: x['NET_AFF_UNITS'].values.sum(), completed[4:]))
    return { "buildings": buildings, "units": units, "netUnits": netUnits, "aff": aff, "netAff": netAff }

def printStats(stats):
    print("Buildings: {}".format(stats["buildings"]))
    print("Net Units: {}".format(stats["netUnits"]))
    print("Total Units: {}".format(sum(stats["netUnits"])))
    print("Net Affordable Units: {}".format(stats["netAff"]))
    print("Total Affordable Units: {}".format(sum(stats["netAff"])))

def contains(data, key):
    for i in range(len(data)):
        if key in data[i].index:
            return True
    return False

In [10]:
completed = []
for i in range(len(data)-1):
    X = data[i]
    X2 = data[i+1]
    X = X[~X.index.isin(X2.index)]
    completed.append(X)

stats = buildStats(completed)
printStats(stats)

Buildings: [18, 15, 28, 48, 19, 17, 16, 24, 71, 27, 34, 43, 50]
Net Units: [1672.0, 328.0, 732.0, 1754.0, 601, 816, 604, 633, 1524, 901, 865, 1185, 977]
Total Units: 12592.0
Net Affordable Units: [160, 10, 34, 167, 64, 96, 175, 116, 71]
Total Affordable Units: 893


# Detection of completed buildings, p.2
Однако, сравнение только со следующим кварталом показало что некоторые строения пропадают и появляются через несколько кварталов, потому мы начали фильтровать со всеми следующими кварталами.

In [11]:
completed = []
for i in range(len(data)-1):
    X = data[i]
    for j in range(i+1,len(data)):
        X2 = data[j]
        X = X[~X.index.isin(X2.index)]
    completed.append(X)

stats = buildStats(completed)
printStats(stats)

Buildings: [18, 15, 28, 43, 18, 16, 16, 24, 71, 27, 34, 41, 50]
Net Units: [1672.0, 328.0, 732.0, 1348.0, 485, 337, 604, 633, 1524, 901, 865, 970, 977]
Total Units: 11376.0
Net Affordable Units: [0, 10, 34, 167, 64, 96, 175, 23, 71]
Total Affordable Units: 640


# Detection of completed buildings, p.4
Однако, тк стройки могут пропадать а потом еще раз появляться то весьма вероятно что некоторые стройки все еще не отфильтрованы. Мы произвели ручную фильтрацию данных и нашли одну большую стройку (57 TEHAMA ST) которая еще не завершена (мы прошлись пешком до них и увидели что они все еще в процессе). Наверняка есть еще.

In [12]:
for i in range(len(completed)):
    if '3736078A' in completed[i].index:
        completed[i] = completed[i].drop('3736078A')

stats = buildStats(completed)
printStats(stats)

Buildings: [18, 15, 28, 43, 18, 16, 16, 24, 71, 27, 34, 41, 50]
Net Units: [1672.0, 328.0, 732.0, 1348.0, 485, 337, 604, 633, 1524, 901, 865, 970, 977]
Total Units: 11376.0
Net Affordable Units: [0, 10, 34, 167, 64, 96, 175, 23, 71]
Total Affordable Units: 640


# Percent of units in top 10 projects for each quartal
Теперь мы посмотрим количество топ 10 завершенных проектов в каждом квартале и то сколько процентов построенных квартир они содержат от общего числа построенных квартир.

In [13]:
for i in range(len(labels)-1):
    topUnits = completed[i].sort_values('NET_UNITS', ascending=False).head(10)['NET_UNITS'].values.sum()
    units = completed[i]['NET_UNITS'].values.sum()
    print(labels[i])
    print(topUnits/units)

2013'Q4
0.995215311005
2014'Q1
0.984756097561
2014'Q2
0.959016393443
2014'Q3
0.940652818991
2014'Q4
0.983505154639
2015'Q1
0.982195845697
2015'Q2
0.990066225166
2015'Q3
0.976303317536
2015'Q4
0.892388451444
2016'Q1
0.977802441731
2016'Q2
0.954913294798
2016'Q3
0.949484536082
2016'Q4
0.939611054248


# Group Stats by Year

In [14]:
for i in range(3):
    yearCount = int(sum(stats['netUnits'][1+i*4:1+i*4+4]))
    print("Year {}: {}".format(2014 + i, yearCount))

Year 2014: 2893
Year 2015: 3098
Year 2016: 3713


# Result Table (top 10)
Как мы видим больше 90% квартир построены в топ-10 проектов, составим отчет по этим строениям за каждый квартал

In [15]:
for i in range(len(labels)-1):
    print(labels[i])
    display(completed[i].sort_values('NET_UNITS', ascending=False).head(10))

2013'Q4


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN
PERMIT_ID,,,,,,
N200607207084,754.0,754.0,0,0,1401 Market St,3507041
N201104224606,315.0,315.0,0,0,185 Channel St,8711023
N201207054130,273.0,273.0,0,0,1155 04th St,8713001
N201012217106,115.0,115.0,0,0,1960-1998 Market St,872005
N200506246051,88.0,88.0,0,0,333 Fremont St,3747019
N200912223711,52.0,52.0,0,0,63 West Point Rd,4624004
N201109074027,24.0,24.0,0,0,1600 Market St,854001
N200701051074,19.0,19.0,0,0,246 Ritch St,3776092
N200912183521,15.0,15.0,0,0,1266 09th Av,1742043


2014'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN
PERMIT_ID,,,,,,
N201106017202,182.0,182.0,0,0,260 05th St,3732008
N201111038205,40.0,40.0,0,0,1501 15th St,3553054
N201110146841,38.0,38.0,0,0,1645-1661 Pacific Av,595013
N200608290880,35.0,35.0,0,0,1080 Sutter St,279011
N201202154236,20.0,20.0,0,0,1717 17th St,3980007
N200711309388,3.0,3.0,0,0,1870 Golden Gate Av,1152017
N201005122282,2.0,2.0,0,0,35 Lloyd St,1260035
N200505031415,2.0,1.0,0,0,530 Sanchez St,3584007
N200901220624,3.0,1.0,0,0,253 Parker Av,1086006


2014'Q2


...

,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN
PERMIT_ID,,,,,,
N201312184508,260,260,31,31,1634 - 1690 PINE ST,0647007
N201209059006,450,191,23,23,55 LAGUNA ST (BLDG 2),0857001
N201207104447,98,98,98,98,1100 GOLDEN GATE AV,0757025
N201207124725,80,80,0,0,LUMINA (PLAZA C),3746001
N201306250465,84,77,11,11,480 POTRERO AV,3973002C
N201306260573,41,41,5,5,450 HAYES ST,0808039
N200912032516,35,35,5,5,1 FRANKLIN ST,0837003
N201406138386,27,20,2,2,832 SUTTER ST,0281003
N201312124038,12,12,0,0,520 09TH ST,3526005


2016'Q3


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN
PERMIT_ID,,,,,,
N201306250394,560,239,0,0,1 HENRY ADAMS ST,3911001
N201409116118,202,202,0,0,1095 MARKET ST,3703059
N201312265046,129,129,0,0,360 BERRY ST,8704004
N201401166475,91,91,0,0,2700 ARELIOUS WALKER DR,4884027
N201211093966,84,81,0,0,2655 BUSH ST,1052024
N201311202405,72,70,11,11,346 POTRERO AV,3962008
N201112070227,470,65,0,0,POTRERO1010 (BLDG 1 OF 2),3833001
N201401297354,21,21,0,0,2353 - 2347 LOMBARD ST,0512026
N201305217457,12,12,1,1,690 PAGE ST,0843016


2016'Q4


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN
PERMIT_ID,,,,,,
N201307303137,416,408,62,62,350 08TH ST,3756003
N201404042522,200,200,0,0,MISSION BAY BLOCK 7,8711031
N201311222660,77,76,0,0,2101 & 2155 WEBSTER ST,0629037
N200809252660,69,69,9,9,1450 FRANKLIN ST,0671006
N201306270646,40,40,0,0,55 LAGUNA ST (AFF SENIOR HOUSING),0857002
N200806164548,76,37,0,0,226 06TH ST,3731003
N200412171712,27,27,0,0,1601 LARKIN ST,0620006
N201412194144,81,24,0,0,690 MARKET ST,0311016
N201401247066,22,22,0,0,233 - 237 SHIPLEY ST,3753096


# Exporting Datasets

In [19]:
for i in range(count):
    fname = 'data/PipelineCleaned_'+labels[i].replace('\'','_')+'.csv'
    data[i].to_csv(fname)
for i in range(count -1 ):
    fname = 'data/PipelineCompleted_'+labels[i].replace('\'','_')+'.csv'
    completed[i].to_csv(fname)